In [1]:
# Day 7 challenge for AOC2019
# https://adventofcode.com/2019/day/7

In [2]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Computer:
    global set_trace
    def __init__(self, size):
        self.size = size  # must specify the memory size
        self.storage = [None] * size  # initialize with None
     
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(set_trace): import pdb; pdb.set_trace()
        if(mode == '0'):
            i = self.storage[mem_location]
            for j in s:
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode (position or immediate)
    def mode_read(self, pos, mode):
        if(set_trace): import pdb; pdb.set_trace()
        # mode: 0 - position, 1 - immediate
        if(mode == '0'):
            return int(self.storage[self.storage[pos]])
        elif(mode == '1'):
            return int(self.storage[pos])

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

    # this function will execute a single sequence (operator + subsequent operands and output location)
    def exec_seq(self, ctr):
        global set_trace
        # ctr - where to start
        # m - the memory object to work from
        # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
        # zero-pad to 5 - most significant 0 is implied
        enhanced_opcode = str(self.read(ctr)).zfill(5)
        # first 3 characters are the parameter modes
        pmodes = enhanced_opcode[:3]
        # last 2 characters are the opcode
        opcode = enhanced_opcode[3:]
        if(set_trace): import pdb; pdb.set_trace()
        # if it is 99, we're done
        if(opcode == '99'):
            return("STOP")
        # this is the "add" operator
        elif(opcode == "01"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(set_trace): import pdb; pdb.set_trace()
            calcval = operand1 + operand2
        # this is the "multiply" operator
        elif(opcode == "02"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            calcval = operand1 * operand2
        # this is the 'store from input' operator
        elif(opcode == "03"):
            if(set_trace): import pdb; pdb.set_trace()
            # get the input
            val = getfrominput()
            # store it in address provided by next code
            self.mode_fill([val], ctr+1, pmodes[2])
            # return next prog counter
            return(ctr+2)
        # this is the 'send to output' operator
        elif(opcode == "04"):
            if(set_trace): import pdb; pdb.set_trace()
            # read the val (based on mode) specified from next code and send to output
            sendtooutput(self.mode_read(ctr+1, pmodes[2]))
            return(ctr+2)
        # this is 'jump if true'
        elif(opcode == "05"):
            if(self.mode_read(ctr+1, pmodes[2])):
                return(self.mode_read(ctr+2, pmodes[1]))
            else:
                return(ctr+3)
        # this is 'jump if false'
        elif(opcode == "06"):
            if(self.mode_read(ctr+1, pmodes[2])):
                return(ctr+3)
            else:
                return(self.mode_read(ctr+2, pmodes[1]))
        # this is 'less than' operator
        elif(opcode == "07"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 < operand2):
                storeval = 1
            else:
                storeval = 0
            self.mode_fill([storeval], ctr+3, pmodes[0])
            return(ctr+4)
        # this is the 'eqals to' operator
        elif(opcode == "08"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 == operand2):
                storeval = 1
            else:
                storeval = 0
            self.mode_fill([storeval], ctr+3, pmodes[0])
            return(ctr+4)        
        # if here, then the opcode is not known
        else:
            return("BADOPCODE")
        # fill the output destination with value depending on mode
        if(set_trace): import pdb; pdb.set_trace()
        self.mode_fill([calcval], ctr+3, pmodes[0])
        # and return the next postion in the memory
        return(ctr+4)

In [3]:
def sendtooutput(s):
    global outputstring
    outputstring = s

In [4]:
def getfrominput():
    # this is the input buffer
    global prog_inputs
    # pop a value off and return it
    return(prog_inputs.pop())

In [5]:
# this will turn on debugging and set breakpoints
set_trace = False

In [6]:
class Amplifier:
    def __init__(self, size):
        self.phase_setting = None
        self.computer = Computer(size)
        pass
    def set_phase_setting(self, setting='0'):
        self.phase_setting = setting

In [7]:
# input buffer - read from by each amp stage
prog_inputs = []

# this is the initial input (read into the input buffer before execution)
outputstring = '0'

# the amp 'settings' - one / stage
amp_settings = [1,0,4,3,2]

# the amp 'program'
seq = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]

In [8]:
# this function returns a list that contains only the unique elements of the input list
def uniquify(seq):
   # Not order preserving    
   retset = set(seq)
   return list(retset)

In [9]:
# this function fills the amp_settings array with only unique elements from the input list
def fill_amp_settings(obj_list):
    global amp_settings
    amp_settings = []
    for i in obj_list:
        for j in obj_list:
            for k in obj_list:
                for l in obj_list:
                    for m in obj_list:
                        pot_list = [i,j,k,l,m]
                        if(len(uniquify(pot_list)) == len(obj_list)):
                            amp_settings.append(pot_list)

In [10]:
# fill the amp_settings array with the unique values from 0 to 4
fill_amp_settings(['0','1','2','3','4'])

In [15]:
# initialize the max
max = 0
for a in amp_settings:
    # initialize the input to the whole system
    outputstring = '0'
    # go through as many stages are as there are settings
    for i in range(0,len(a)):
        # instantiate the amp
        amp = Amplifier(len(seq))
        # load the program
        amp.computer.fill(seq)
        # set the the inputs
        # the input to this stage is the output from the last stage (persisted in outputstring)
        prog_inputs.append(outputstring)
        # put this stage's setting onto the input buffer
        prog_inputs.append(a[i])
        # run the program
        # execute the program
        ctr = 0
        ret = amp.computer.exec_seq(ctr)
        while(ret != "STOP" and ret != "BADOPCODE"):
            ret = amp.computer.exec_seq(ret)
        # print("PROG STATE AFTER AMP#{}: {}".format(i,ret))
        if(set_trace): import pdb; pdb.set_trace()

    # print("final output: {}".format(outputstring))
    outnum = int(outputstring)
    if(outnum > max):
        max = outnum
        s = a

In [16]:
print("settings: {} produces a max thruster output of: {}".format(s,max))

settings: ['1', '0', '4', '3', '2'] produces a max thruster output of: 65210
